Reference [A Guide to Gradient Boosted Trees with XGBoost in Python](https://jessesw.com/XG-Boost/)
Check the implementation with XGBoost in [XGBoost_learning - v2](../GBM/xgBoost/XGBoost_learning - v2.ipynb)

After some parameter tuning stuff, XGBoost gives **0.8687** for the test set.
Now we try the same dataset with random forest.

In [3]:
import numpy as np
import pandas as pd

In [4]:
train_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header = None)
test_set = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test',
                      skiprows = 1, header = None) # Make sure to skip a row for the test set

In [5]:
train_set.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
test_set.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [7]:
col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']

In [8]:
test_set.columns = col_labels
train_set.columns = col_labels

In [9]:
train_set.info() # note that Pandas will not take '?' as nan automatically, use na_values to specify it
# By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, 
# ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
wage_class        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Remove rows containing unknown values (" ?")

In [10]:
train_set = train_set.replace(" ?", np.nan).dropna()
test_set = test_set.replace(" ?", np.nan).dropna()
print(train_set.shape, test_set.shape)

(30162, 15) (15060, 15)


In [11]:
test_set.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.


In [12]:
# note that the wage_class in the test set has an additional dot after its value
test_set['wage_class'] = test_set['wage_class'].replace({' <=50K.': ' <=50K', ' >50K.': ' >50K'})
test_set['wage_class'].unique()

array([' <=50K', ' >50K'], dtype=object)

## Applying Ordinal Encoding to Categoricals
All called numeric encoding. That is, assign a unique number to each category. 

In [13]:
combine_set = pd.concat([train_set, test_set])

In [14]:
for feature in combine_set.columns:
    if combine_set[feature].dtype == 'object': # 'category'
        combine_set[feature] = pd.Categorical(combine_set[feature]).codes # code each category from 0

In [15]:
train_set = combine_set[0:train_set.shape[0]]
test_set = combine_set[train_set.shape[0]:]
train_y = train_set.pop('wage_class')
test_y = test_set.pop('wage_class')
train_X = train_set;
test_X = test_set;

In [16]:
# check the class balance
train_y.value_counts()

0    22654
1     7508
Name: wage_class, dtype: int64

In [17]:
test_y.value_counts() # as we can see, the training and test set are imbalanced

0    11360
1     3700
Name: wage_class, dtype: int64

# Random Forest: parameter tuning
Reference: [Tuning the parameters of your Random Forest model](https://www.analyticsvidhya.com/blog/2015/06/tuning-random-forest-model/)

We mainly tune only one parameter: max_features, i.e., the number of features randomly chosen to test for node splitting. Also known as *random subspace*. 

Increase max_features, we can get a stronger classifier with smaller bias. However, we will also increase the correlation among individual trees.

In the original paper on random forests, it was shown that the forest error rate depends on two things:
+ The correlation between any two trees in the forest. Increasing the correlation increases the forest error rate. (diversity)
+ The strength of each individual tree in the forest. A tree with a low error rate is a strong classifier. Increasing the strength of the individual trees decreases the forest error rate.

Reccomended: $\sqrt{M}$ for classification, where $M$ is the total number of features.

## 1. Initial parameter values

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
# defaut values for the parameters
RandomForestClassifier().get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

**out of bag (oob) errors**
This is a random forest cross validation method. It is very similar to leave one out validation technique, however, this is so much faster. This method simply tags every observation used in different tress. And then it finds out a maximum vote score for every observation based on only trees which did not use this particular observation to train itself.

**max_feature** (default=”auto”, that is, $\sqrt{M}$)
We first try the default parameters

In [20]:
rf = RandomForestClassifier(n_estimators=300, oob_score=True, n_jobs=-1)
rf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [22]:
rf.oob_score_ # Score of the training dataset obtained using an out-of-bag estimate.

0.85282806179961546

In [24]:
from sklearn.metrics import accuracy_score
pred_y = rf.predict(test_X) # unlike XGBoost, which gives the probability of p(y=1|x), RandomForestClassifer gives the label directly
accuracy_score(pred_y, test_y)

0.84920318725099597

As we see, with the default configuration, RF only gives **0.8492** on the test set.

## 2. Tune parameters with grid search
As we have mentioned, the most important parameters should be max_features and n_estimators

In [29]:
print(train_X.shape, np.sqrt(train_X.shape[1]), np.log2(train_X.shape[1]))

(30162, 14) 3.74165738677 3.80735492206


In [32]:
rf.set_params(oob_score=False)
rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 300,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [33]:
param_grid = {'n_estimators': [100, 300, 500, 700], 'max_features': list(range(1, 15, 2))}
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(rf, param_grid, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True) # last argument to avoid warning
gs.fit(train_X, train_y)
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_max_features,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
7,19.806208,2.828817,0.854519,0.999975,3,700,"{'max_features': 3, 'n_estimators': 700}",1,0.852312,1.000000,...,0.854964,0.999959,0.859914,1.000000,0.856906,0.999959,0.581446,1.027329,0.003901,0.000020
6,12.271001,2.225607,0.854154,0.999975,3,500,"{'max_features': 3, 'n_estimators': 500}",2,0.853804,1.000000,...,0.852478,0.999959,0.861074,1.000000,0.855414,0.999959,1.190391,0.874111,0.004248,0.000020
5,7.905346,2.941790,0.854022,0.999975,3,300,"{'max_features': 3, 'n_estimators': 300}",3,0.850986,1.000000,...,0.851318,0.999959,0.859582,1.000000,0.857072,0.999959,1.578887,0.162603,0.003605,0.000020
9,10.635302,3.375983,0.853988,0.999975,5,300,"{'max_features': 5, 'n_estimators': 300}",4,0.853473,1.000000,...,0.853804,0.999959,0.857095,1.000000,0.856243,0.999959,0.957062,0.372298,0.002711,0.000020
10,17.244863,3.134020,0.853790,0.999975,5,500,"{'max_features': 5, 'n_estimators': 500}",5,0.852147,1.000000,...,0.854136,0.999959,0.857593,1.000000,0.854750,0.999959,0.517955,1.241880,0.002459,0.000020
4,4.254277,1.729370,0.853657,0.999950,3,100,"{'max_features': 3, 'n_estimators': 100}",6,0.852810,0.999959,...,0.853307,0.999959,0.857261,1.000000,0.855745,0.999917,0.811008,0.294376,0.002771,0.000031
19,40.222703,4.219654,0.853259,0.999975,9,700,"{'max_features': 9, 'n_estimators': 700}",7,0.847671,1.000000,...,0.857285,0.999959,0.858090,1.000000,0.853756,0.999959,2.832295,0.445413,0.004128,0.000020
15,34.202232,2.928781,0.853193,0.999975,7,700,"{'max_features': 7, 'n_estimators': 700}",8,0.848831,1.000000,...,0.856290,0.999959,0.856432,1.000000,0.855745,0.999959,0.989151,0.891732,0.003637,0.000020
13,13.891595,3.376584,0.853193,0.999975,7,300,"{'max_features': 7, 'n_estimators': 300}",8,0.851484,1.000000,...,0.855130,0.999959,0.856267,1.000000,0.853258,0.999959,1.234467,0.309073,0.002344,0.000020
11,27.080014,2.873443,0.853126,0.999975,5,700,"{'max_features': 5, 'n_estimators': 700}",10,0.850986,1.000000,...,0.854301,0.999959,0.856930,1.000000,0.853424,0.999959,0.458685,0.857003,0.002463,0.000020


It seems max_features=3 gives the best result, which is very close to $\sqrt{M}$.
Besides, we find that as n_estimators increase, the test_score also increases.

In [34]:
rf = gs.best_estimator_
rf.get_params()

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 3,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 700,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

We now tune the n_estimators.

In [35]:
param_grid = {'n_estimators': [700, 800, 900, 1000]}
gs = GridSearchCV(rf, param_grid, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True) # last argument to avoid warning
gs.fit(train_X, train_y)
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,15.869040,4.447222,0.854320,0.999975,700,{'n_estimators': 700},1,0.853307,1.0,0.847671,...,0.852478,0.999959,0.862235,1.0,0.855911,0.999959,5.311613,0.875635,0.004771,0.00002
2,29.073641,3.998681,0.854320,0.999975,900,{'n_estimators': 900},1,0.853141,1.0,0.848334,...,0.853804,0.999959,0.859914,1.0,0.856409,0.999959,1.129420,0.339088,0.003825,0.00002
3,26.100556,2.367647,0.854022,0.999975,1000,{'n_estimators': 1000},3,0.852478,1.0,0.848168,...,0.852478,0.999959,0.859251,1.0,0.857735,0.999959,4.285918,1.061071,0.004004,0.00002
1,23.304639,3.634539,0.853259,0.999975,800,{'n_estimators': 800},4,0.851152,1.0,0.848500,...,0.851981,0.999959,0.859251,1.0,0.855414,0.999959,1.958960,1.188857,0.003722,0.00002


Because train_score is close to 100% and much larger than the test_score, we suspect overfitting happens. 

We may control the tree complexity by max_depth or min_samples_split. However, we have no idea what the best depth may be. Therefore, we tune the min_samples_split, whose default is 2.

In [36]:
rf = gs.best_estimator_
param_grid = {'min_samples_split': [2, 10, 20, 50]}
gs = GridSearchCV(rf, param_grid, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True) # last argument to avoid warning
gs.fit(train_X, train_y)
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_min_samples_split,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
2,19.902845,3.224243,0.860918,0.914586,20,{'min_samples_split': 20},1,0.856788,0.914377,0.858445,...,0.859108,0.914418,0.867871,0.915665,0.862378,0.912934,0.893551,0.731043,0.003922,0.000987
3,14.231904,1.183823,0.860354,0.889555,50,{'min_samples_split': 50},2,0.856125,0.888599,0.855793,...,0.860269,0.890298,0.867208,0.889184,0.862378,0.889478,3.446976,0.789921,0.004237,0.000640
1,19.727523,3.546968,0.858995,0.941027,10,{'min_samples_split': 10},3,0.856456,0.940777,0.856125,...,0.858445,0.940652,0.866048,0.942354,0.857901,0.939953,2.017428,0.996953,0.003631,0.000807
0,15.672002,4.420475,0.853524,0.999975,2,{'min_samples_split': 2},4,0.851484,1.000000,0.846677,...,0.852975,0.999959,0.859582,1.000000,0.856906,0.999959,5.223379,1.063499,0.004461,0.000020


Wow, it seems *min_sample_split* gives a best result. Since the above test is very coarse, we now try fine tuning of *min_sample_split*.

In [37]:
rf = gs.best_estimator_
param_grid = {'min_samples_split': [15, 20, 25, 30, 40, 50, 60]}
gs = GridSearchCV(rf, param_grid, scoring='accuracy', cv=5, n_jobs=-1, return_train_score=True) # last argument to avoid warning
gs.fit(train_X, train_y)
pd.DataFrame(gs.cv_results_).sort_values('mean_test_score', ascending=False)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_min_samples_split,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,...,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
2,18.663659,3.273856,0.861249,0.907781,25,{'min_samples_split': 25},1,0.856622,0.906668,0.859937,...,0.858279,0.908285,0.868700,0.908620,0.862709,0.906883,0.341421,0.639800,0.004233,0.000831
4,18.254677,2.513031,0.861050,0.895025,40,{'min_samples_split': 40},2,0.855130,0.893489,0.857616,...,0.860103,0.895603,0.868203,0.895400,0.864202,0.894907,1.313882,0.518387,0.004665,0.000817
3,17.421503,3.635205,0.860918,0.902070,30,{'min_samples_split': 30},3,0.856290,0.900825,0.857285,...,0.860103,0.902773,0.867208,0.902611,0.863704,0.901910,1.313005,1.119533,0.004065,0.000691
1,17.891027,3.229525,0.860752,0.914918,20,{'min_samples_split': 20},4,0.856622,0.914543,0.857782,...,0.860434,0.914045,0.868203,0.916121,0.860720,0.914218,1.900287,1.301359,0.004037,0.000824
5,16.934968,3.738776,0.860387,0.889290,50,{'min_samples_split': 50},5,0.855627,0.887936,0.855627,...,0.860434,0.890298,0.866877,0.889059,0.863373,0.889395,1.364512,0.715494,0.004390,0.000792
6,15.050470,1.222542,0.860321,0.885833,60,{'min_samples_split': 60},6,0.854964,0.885283,0.855627,...,0.860932,0.886775,0.866545,0.885495,0.863538,0.885376,4.634490,0.884745,0.004476,0.000579
0,13.481889,3.954725,0.859658,0.925005,15,{'min_samples_split': 15},7,0.855130,0.924365,0.856788,...,0.859937,0.924406,0.866545,0.926482,0.859891,0.924247,4.827555,0.757872,0.003907,0.000871


# Test set performance

In [41]:
rf = gs.best_estimator_
pred_y = rf.predict(test_X)
accuracy_score(pred_y, test_y)

0.85909694555112881

# Summary
+ We mainly tuned *max_features*, *n_estimators* and *min_samples_split*. 
+ Usually $\sqrt{M}$ is a good candidate for *max_features*.
+ RF is not sensitive to *n_estimators*, we can choose a large number, around 500, for example.
+ The default 2 for *min_samples_split* may be too small for a moderate dataset, which may lead to some level of overfitting.
+ The best test set accurary is 0.8591, still smaller than 0.8687 with XGBoost.
+ Simply with the default settings, RF gives 0.8492.